In [8]:
!pip install selenium
!pip install webdriver-manager

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time
import pandas as pd

In [10]:
# --- Setup Chrome WebDriver ---
options = Options()
options.add_argument("--headless")  # Run in background
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

In [12]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# --- Open cafef.vn homepage ---
url = "https://cafef.vn/trang-chu.chn"
driver.get(url)
time.sleep(3)  # Wait for content to load

# --- Đợi phần tin xuất hiện ---
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.tlitem"))
    )
except:
    print("Không tìm thấy bài viết nào.")
    driver.quit()
    exit()

# --- Lấy danh sách bài viết ---
articles = driver.find_elements(By.CSS_SELECTOR, "div.tlitem")

data = []
for a in articles:
    try:
        title_elem = a.find_element(By.CSS_SELECTOR, "h3 a")
        title = title_elem.text.strip()
        link = title_elem.get_attribute("href")

        desc = ""
        try:
            desc_elem = a.find_element(By.CSS_SELECTOR, "h3 + div")
            desc = desc_elem.text.strip()
        except:
            pass

        data.append({
            "title": title,
            "link": link,
            "description": desc
        })
    except Exception as e:
        print("Lỗi parse:", e)

# --- Lưu ra CSV ---
df = pd.DataFrame(data)
df.to_csv("cafef_news.csv", index=False, encoding="utf-8-sig")

print(f"✅ Đã crawl được {len(df)} bài viết.")
driver.quit()

✅ Đã crawl được 5 bài viết.
